# NLP Analysis of Scientific Abstracts
---
####  US National Library of Medicine National Institutes of Health 
#### The National Center for Biotechnology Information 
#### PubMed

In [1]:
# Import custom module for calling abstracts from a key word or phrase
from lib.get_abstracts import get_abstracts

In [2]:
# Import standard analysis tools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [13]:
# Import tools for NLP 
import scipy
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import LatentDirichletAllocation as LDA
from sklearn.decomposition import NMF

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import pyLDAvis
import pyLDAvis.gensim

In [4]:
# Get abstracts and create csv of abstracts
# API key obtained from NCBI
#  get_abstracts('gene therapy', 1000, 'ragorham1@gmail.com', 
#                '3ffbbb6bd110815d69e4aa14b7c26d72ab09')

In [5]:
# Process abstracts 
df = pd.read_csv('./data/gene therapy.csv', header=-1, 
                 names=['pmid', 'abstract'])
df = df.dropna()

abstracts = list(df.abstract)

# Make tf-idf matrix
vectorizer = TfidfVectorizer(stop_words='english')
abstracts_tfidf = vectorizer.fit_transform(abstracts)

In [6]:
#  Make the wors list
terms = vectorizer.get_feature_names()

# Number of topics
ntopics=10

# Linking words to topics
def word_topic(tfidf,solution, wordlist):
    
    # Loading scores for each word on each topic/component.
    words_by_topic=tfidf.T * solution

    # Linking the loadings to the words in an easy-to-read way.
    components=pd.DataFrame(words_by_topic,index=wordlist)
    
    return components
# Extracts the top N words and their loadings for each topic.
def top_words(components, n_top_words):
    n_topics = range(components.shape[1])
    index= np.repeat(n_topics, n_top_words, axis=0)
    topwords=pd.Series(index=index)
    for column in range(components.shape[1]):
        # Sort the column so that highest loadings are at the top.
        sortedwords=components.iloc[:,column].sort_values(ascending=False)
        # Choose the N highest loadings.
        chosen=sortedwords[:n_top_words]
        # Combine loading and index into a string.
        chosenlist=chosen.index +" "+round(chosen,2).map(str) 
        topwords.loc[column]=chosenlist
    return(topwords)

# Number of words to look at for each topic.
n_top_words = 10

In [7]:
svd= TruncatedSVD(ntopics)
lsa = make_pipeline(svd, Normalizer(copy=False))
abstracts_lsa = lsa.fit_transform(abstracts_tfidf)

components_lsa = word_topic(abstracts_tfidf, abstracts_lsa, terms)

topwords=pd.DataFrame()
topwords['LSA']=top_words(components_lsa, n_top_words)    

In [8]:
lda = LDA(n_topics=ntopics, 
          doc_topic_prior=None, # Prior = 1/n_documents
          topic_word_prior=1/ntopics,
          learning_decay=0.7, # Convergence rate.
          learning_offset=10.0, # Causes earlier iterations to have less influence on the learning
          max_iter=10, # when to stop even if the model is not converging (to prevent running forever)
          evaluate_every=-1, # Do not evaluate perplexity, as it slows training time.
          mean_change_tol=0.001, # Stop updating the document topic distribution in the E-step when mean change is < tol
          max_doc_update_iter=100, # When to stop updating the document topic distribution in the E-step even if tol is not reached
          n_jobs=-1, # Use all available CPUs to speed up processing time.
          verbose=0, # amount of output to give while iterating
          random_state=0
         )

abstracts_lda = lda.fit_transform(abstracts_tfidf) 

components_lda = word_topic(abstracts_tfidf, abstracts_lda, terms)

topwords['LDA']=top_words(components_lda, n_top_words)

/usr/local/lib/python3.6/dist-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


In [9]:
nmf = NMF(alpha=0.0, 
          init='nndsvdar', # how starting value are calculated
          l1_ratio=0.0, # Sets whether regularization is L2 (0), L1 (1), or a combination (values between 0 and 1)
          max_iter=200, # when to stop even if the model is not converging (to prevent running forever)
          n_components=ntopics, 
          random_state=0, 
          solver='cd', # Use Coordinate Descent to solve
          tol=0.0001, # model will stop if tfidf-WH <= tol
          verbose=0 # amount of output to give while iterating
         )
abstracts_nmf = nmf.fit_transform(abstracts_tfidf) 

components_nmf = word_topic(abstracts_tfidf, abstracts_nmf, terms)

topwords['NNMF']=top_words(components_nmf, n_top_words)

In [10]:
# for topic in range(ntopics):
#     print('Topic {}:'.format(topic))
#     print(topwords.loc[topic])

In [11]:
import pyLDAvis.sklearn
 
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda, abstracts_tfidf, vectorizer, R=10, mds='tsne')
panel

PreparedData(topic_coordinates=            Freq  cluster  topics           x           y
topic                                                    
8      87.601462        1       1 -379.869904   -2.430183
7       1.446706        1       2 -230.104340 -225.089737
4       1.421184        1       3 -133.869019   32.119122
1       1.407821        1       4 -297.173553  252.858292
5       1.404259        1       5  331.733765   97.406982
9       1.394102        1       6  249.019531 -157.868271
0       1.361428        1       7   85.722816   62.927479
3       1.331054        1       8  -58.700191  294.329041
6       1.327780        1       9   10.548111 -199.251846
2       1.304203        1      10  181.978317  320.082550, topic_info=      Category       Freq              Term      Total  loglift  logprob
term                                                                    
1416   Default   2.000000           article   2.000000  10.0000  10.0000
6266   Default   3.000000               ibd   3.000000   9.0000   9.0000
7783   Default   2.000000          melanoma   2.000000   8.0000   8.0000
4912   Default   3.000000            family   3.000000   7.0000   7.0000
12642  Default   2.000000   transplantation   2.000000   6.0000   6.0000
12691  Default   3.000000            trials   3.000000   5.0000   5.0000
3907   Default   3.000000         disorders   3.000000   4.0000   4.0000
2291   Default   3.000000            caused   3.000000   3.0000   3.0000
13200  Default   3.000000             virus   3.000000   2.0000   2.0000
11776  Default   2.000000             stage   2.000000   1.0000   1.0000
2152    Topic1  19.147899            cancer  19.208907   0.1292  -5.7085
2382    Topic1  18.151540             cells  18.213050   0.1290  -5.7620
2379    Topic1  16.173395              cell  16.234321   0.1286  -5.8774
5404    Topic1  15.464674              gene  15.526293   0.1284  -5.9222
9295    Topic1  14.715094          patients  14.776528   0.1282  -5.9719
12375   Topic1  14.374279           therapy  14.435752   0.1281  -5.9953
12673   Topic1  12.538123         treatment  12.599214   0.1275  -6.1320
3897    Topic1  11.744774           disease  11.806005   0.1272  -6.1973
4827    Topic1  11.635399        expression  11.696561   0.1271  -6.2067
12763   Topic1  10.464082             tumor  10.524902   0.1266  -6.3128
8742    Topic2   0.060381                ns   0.187782   3.1013  -7.3643
11841   Topic2   0.053239           steroid   0.180670   3.0140  -7.4902
3304    Topic2   0.039972               csm   0.168438   2.7975  -7.7768
11752   Topic2   0.033660              srns   0.161298   2.6689  -7.9487
10416   Topic2   0.026036             radix   0.153453   2.4620  -8.2055
3148    Topic2   0.026182              coq6   0.154678   2.4596  -8.1999
11086   Topic2   0.025335            rs5333   0.153180   2.4365  -8.2328
4432    Topic2   0.025319             endra   0.153123   2.4362  -8.2334
1512    Topic2   0.023083              atcc   0.153866   2.3389  -8.3259
1899    Topic2   0.022930      biosynthetic   0.155614   2.3210  -8.3325
...        ...        ...               ...        ...      ...      ...
7058    Topic8   0.028311            kabuki   0.157035   2.6060  -8.0384
11162   Topic8   0.026478       sarcoidosis   0.161638   2.5101  -8.1053
5736    Topic8   0.020719            hailey   0.149479   2.3431  -8.3506
5428    Topic8   0.020671    genodermatosis   0.149710   2.3392  -8.3529
8059    Topic8   0.017354              mll2   0.146390   2.1868  -8.5278
9773    Topic8   0.017319  polymalformative   0.146683   2.1827  -8.5298
12557   Topic8   0.017311     traditionally   0.146751   2.1818  -8.5303
6118    Topic8   0.013591            hspca1   0.141858   1.9738  -8.7722
1533    Topic8   0.017473            atpase   0.242348   1.6895  -8.5210
12703   Topic9   0.033770      triglyceride   0.192304   2.5822  -7.8596
12540   Topic9   0.024993              tpmt   0.154342   2.5011  -8.1606
12384   Topic9   0.024980        thiopurine   0.1